In [13]:
from caveclient import CAVEclient, auth
from caveclient import chunkedgraph as cg
import pandas as pd
import numpy as np
from cloudvolume import CloudVolume
import gpytoolbox as gpy


In [ ]:
global_url = "https://global.brain-wire-test.org/"
local_url = "https://pcgv3local.brain-wire-test.org"
datastack = "fish1_full"
dataset = "fish1_v250915"
resolution = (16, 16, 30)
# Set bounds of the region of interest
TOP_LEFT_COORDS = [7000, 2000, 0]
BOTTOM_RIGHT_COORDS = [54000, 30000, 8735]

In [37]:
client = CAVEclient(datastack_name=datastack, server_address=global_url)
cggraph = cg.ChunkedGraphClient(server_address=local_url, table_name=dataset, auth_client=auth.AuthClient(token=client.auth.token))

In [38]:
cv_seg = CloudVolume("precomputed://gs://fish1-public/seg_241003_agg241003", cache=True)
cv_cave = client.info.segmentation_cloudvolume()

In [95]:
def interior_point_fast(V_vox, F, lo, hi, ngrid=(1, 1, 1), nrand=200000,
                        threshold=0.5, use_abs=False, seed=123):
    rng = np.random.default_rng(seed)

    def inside_idx(P):
        W = gpy.fast_winding_number(P, V_vox, F)
        return np.where((np.abs(W) if use_abs else W) >= threshold)[0]

    # coarse grid
    lin = [np.linspace(lo[i], hi[i], ngrid[i]) for i in range(3)]
    GX,GY,GZ = np.meshgrid(*lin, indexing="xy")
    P = np.column_stack([GX.ravel(), GY.ravel(), GZ.ravel()])
    idx = inside_idx(P)
    if idx.size:
        return P[idx[0]]

    # random
    P = rng.uniform(lo, hi, size=(nrand,3))
    idx = inside_idx(P)
    if idx.size:
        p = P[idx[0]]
        # refine around the found point
        for _ in range(3):
            span = (hi - lo) * 0.1
            lo2 = np.maximum(p - span, lo)
            hi2 = np.minimum(p + span, hi)
            P2 = rng.uniform(lo2, hi2, size=(50000,3))
            idx2 = inside_idx(P2)
            if idx2.size:
                p = P2[idx2[0]]
        return p

    raise RuntimeError("No interior point found; mesh may be open or outside bounds.")


In [ ]:
def get_cave_id_from_segmentation_id(segmentation_ids):
    cave_ids = []
    supervoxel_ids = []
    failed_segmentation_ids = []
    for segmentation_id in segmentation_ids:
        try:
            mesh = cv_seg.mesh.get([segmentation_id], lod=2)
            vertices = np.array(mesh[segmentation_id].vertices, dtype=np.float64)
            faces = np.array(mesh[segmentation_id].faces, dtype=np.int32)

            v_vox = vertices / np.array(resolution)
            mn = v_vox.min(axis=0); mx = v_vox.max(axis=0)
            lo = np.maximum(mn, TOP_LEFT_COORDS)
            hi = np.minimum(mx, BOTTOM_RIGHT_COORDS)

            p_vox = interior_point_fast(v_vox, faces, lo, hi, \
                threshold=0.99, use_abs=False, seed=np.random.randint(0, 100))
            
            # get root id
            supervoxel_id = cv_cave.download_point(pt=p_vox, 
                                        size=1, 
                                        coord_resolution=resolution)
            supervoxel_id = np.int64(supervoxel_id[0, 0, 0, 0]) 
            supervoxel_ids.append(supervoxel_id)
            try:
                root_id = cggraph.get_root_id(supervoxel_id)
                cave_ids.append(root_id)
            except Exception as e:
                root_id = 0
                print(f"Failed to find segmentation for {segmentation_id} in CAVE's Segmentation Volume")
                failed_segmentation_ids.append(segmentation_id)
        except Exception as e:
            print(f"Unable to find mesh for {segmentation_id} in Segmentation Volume")
            failed_segmentation_ids.append(segmentation_id)

        print(segmentation_id, p_vox, supervoxel_id, root_id)

    return cave_ids, supervoxel_ids, failed_segmentation_ids



In [ ]:
segmentation_ids = ["list your segmentation ids here"]

In [ ]:
cave_ids, supervoxel_ids, failed_segmentation_ids = get_cave_id_from_segmentation_id(segmentation_ids)

In [ ]:
# Copy and paste the following supervoxel_ids into Neuroglancer, The neuroglancer will load latest root segments by supoervoxel_ids.
supervoxel_ids